In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import timedelta, date
from os.path import join
from unidecode import unidecode

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.utils.normalize import normalize_text, normalize_hash
from bulletin.systems.sim import Sim
from bulletin.systems.sivep import Sivep
from bulletin.systems.notifica import Notifica

In [ ]:
sim = Sim()
# sim.load()
sim.read_all_database_files()
sim.fix_dtypes()
sim.save(replace=True)

In [ ]:
cbo_dora = sim.tables['cbo_dora']
sim.df = sim.df.loc[sim.df['OCUP'].isin(cbo_dora['id'].values)]

In [ ]:
sim.to_notifica()
sim_covid = sim.get_obitos()

In [ ]:
del sim

In [ ]:
sivep = Sivep()
sivep.load('sraghospitalizado')
# sivep.read_all_database_files()
# sivep.fix_dtypes()
# sivep.save(replace=True)

In [ ]:
sivep.to_notifica()
sivep_covid = sivep.get_obitos()

In [ ]:
del sivep

In [ ]:
# ocupacoes_cbo = pd.read_csv('C:/Users/est.joaosilva/Documents/1106/CBO2002 - Ocupacao.csv', sep=';', encoding='latin-1').rename(columns={'TITULO':'OCUPACAO'})
# ocupacoes_cbo['OCUPACAO'] = ocupacoes_cbo['OCUPACAO'].apply(lambda x: normalize_text(x))

In [ ]:
# ocupacoes_dora = pd.read_excel('C:/Users/est.joaosilva/Documents/1106/LISTA_OCUPACOES_DORA.xlsx')
# ocupacoes_dora['OCUPACAO'] = ocupacoes_dora['OCUPACAO'].apply(lambda x: normalize_text(x))
# ocupacoes_dora = ocupacoes_dora.merge(how = 'left', on = 'OCUPACAO', right = ocupacoes_cbo)
# ocupacoes_dora['CODIGO'] = ocupacoes_dora['CODIGO'].astype('str').str.zfill(6)

In [ ]:
# ocupacoes_dora['FAMILIA'] = ocupacoes_dora['CODIGO'].apply(lambda x: x[:4])
# ocupacoes_dora['CODIGO'] = ocupacoes_dora['CODIGO'].astype('int')
# lista_cbo = ocupacoes_dora['CODIGO'].unique()

In [ ]:
# ocupacoes_dora[['CODIGO', 'OCUPACAO', 'FAMILIA']].rename(columns={'CODIGO':'id', 'OCUPACAO':'ocupacao', 'FAMILIA':'familia'}).to_csv('cbo_dora.csv', index=False, encoding='utf-8-sig')

In [ ]:
#sim_covid = sim_covid.loc[sim_covid['cod_cbo'].isin(lista_cbo)]

In [ ]:
# sim_covid = sim_covid.loc[
#     sim_covid['hash_mae'].isin(sivep_covid['hash_mae']) |
#     sim_covid['hash_nasc'].isin(sivep_covid['hash_nasc']) |
#     sim_covid['hash_resid'].isin(sivep_covid['hash_resid']) |
#     sim_covid['hash_atend'].isin(sivep_covid['hash_atend'])
# ]

In [ ]:
# sim_covid.shape[0]

In [ ]:
notifica = Notifica()
notifica.load('notifica',compress=False)

In [ ]:
saude_notifica = notifica.df.loc[(notifica.df['ocupacao'].isin([1,3]))].copy()
saude_notifica.shape[0]

In [ ]:
cbo = notifica.tables['cod_cbo'][['id', 'value']].rename(columns={'id':'cod_cbo', 'value':'cbo_descricao'}).copy()
del notifica

In [ ]:
saude = saude_notifica.loc[saude_notifica['uf_residencia'] == 41]
saude = saude.loc[((saude['status_notificacao'] == 1) | (saude['status_notificacao'] == 2))]
saude = saude.loc[saude['excluir_ficha'] == 2]
saude.shape

In [ ]:
saude.loc[(~(saude['evolucao'] == 2)) & (
    saude['hash_mae'].notna() & saude['hash_mae'].isin(sim_covid['hash_mae']) |
    saude['hash_nasc'].notna() & saude['hash_nasc'].isin(sim_covid['hash_nasc'])
), 'evolucao'] = 2

In [ ]:
sivep_saude = sivep_covid.loc[(
    sivep_covid['hash_mae'].notna() & sivep_covid['hash_mae'].isin(sim_covid['hash_mae']) |
    sivep_covid['hash_nasc'].notna() & sivep_covid['hash_nasc'].isin(sim_covid['hash_nasc'])
)].copy()

In [ ]:
saude = saude.append(sivep_saude, ignore_index=True)
saude.shape[0]

In [ ]:
saude = pd.merge(how='left', left=saude, on = 'cod_cbo', right = cbo)

In [ ]:
#NAAAAAAAAAO ENTENDIIIIIIIIIIIIIII
#saude.loc[saude['id'].notna()].groupby('cod_cbo')[['paciente']].count().sort_values('paciente')
#saude.loc[saude['cbo_descricao'].isnull()][['cod_cbo', 'cbo_descricao']]

In [ ]:
saude.loc[saude['cod_cbo'].isnull(), 'cod_cbo'] = 0
saude.loc[(saude['cbo_descricao'].isnull()) | (saude['cod_cbo']==0), 'cbo_descricao'] = 'Não informado'
saude.loc[saude['evolucao'] == 5, 'evolucao'] = 2

In [ ]:
# saude.loc[(saude['cbo_descricao'].isnull()), 'cbo_descricao'] = 'Não informado'
# saude.loc[(saude['cod_cbo'].isin([0, -1])), 'cod_cbo'] = 0
# del saude['hash_descricao_ocupacao']
# saude[['id', 'cns', 'cpf', 'paciente', 'nome_mae', 'data_nascimento', 'ibge_residencia', 'uf_residencia', 'ocupacao', 'cod_cbo', 'cbo_descricao', 'ocupacao_descricao', 'data_notificacao', 'data_cadastro', 'data_coleta', 'data_recebimento', 'data_liberacao', 'data_diagnostico', 'evolucao', 'data_cura_obito', 'metodo', 'classificacao_final', 'status_notificacao', 'excluir_ficha']].to_excel(join(default_output, 'semanais', 'EXPORTACAO_trab_saude.xlsx'), index= False)
#saude.to_csv(join(default_output, 'semanais', 'EXPORTACAO_trab_saude.csv'), index= False, encoding = 'utf-8-sig', sep = ';')

In [ ]:
# saude.loc[(~saude['evolucao'] == 2) & (
#     saude['hash_mae'].isin(sim_covid['hash_mae']) |
#     saude['hash_nasc'].isin(sim_covid['hash_nasc']) |
#     saude['hash_resid'].isin(sim_covid['hash_resid']) |
#     saude['hash_atend'].isin(sim_covid['hash_atend'])
# ), 'evolucao'] = 2

In [ ]:
saude.loc[~(saude['ocupacao_descricao'].isnull()), 'hash_descricao_ocupacao'] = saude.loc[~(saude['ocupacao_descricao'].isnull()), 'ocupacao_descricao'].apply(lambda row: unidecode(row))
saude.loc[~(saude['ocupacao_descricao'].isnull()), 'hash_descricao_ocupacao'] = saude.loc[~(saude['ocupacao_descricao'].isnull()), 'hash_descricao_ocupacao'].apply(lambda row: normalize_hash(row))

In [ ]:
#2251 - MÉDICO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('MEDICO'))), 'cod_cbo'] = 2251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('MEDICA'))), 'cod_cbo'] = 2251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('OTORRINO'))), 'cod_cbo'] = 2251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('OFTAMOLOGISTA'))), 'cod_cbo'] = 2251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ENDOSCOPISTA'))), 'cod_cbo'] = 2251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ANESTESI'))), 'cod_cbo'] = 2251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('225'))), 'cod_cbo'] = 2251
#3222 - AUXILIAR DE ENFERMAGEM
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECEMENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXEMENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECDEENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECDEENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOEMENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICAEMENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICAENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICODEENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICADEENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDEENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIAREMENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDEENFERMAGEM'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECENF'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXENF'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECDEENF'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDEENF'))), 'cod_cbo'] = 3222
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('3222'))), 'cod_cbo'] = 3222
#2235 - ENFERMEIRO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ENFERMEIRA'))), 'cod_cbo'] = 2235
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ENFERMEIRO'))), 'cod_cbo'] = 2235
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2235'))), 'cod_cbo'] = 2235
#3251 - AUXILIAR DE FARMACIA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ASSISTENTEDEFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECEMFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXEMFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECDEFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECDEFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOEMFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICAEMFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICAFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICODEFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICADEFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDEFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIAREMFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARFARMACIA'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECFARM'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXFARM'))), 'cod_cbo'] = 3251
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('3251'))), 'cod_cbo'] = 3251
#2234 - FARMACEUTICO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('FARMACEUTICO'))), 'cod_cbo'] = 2234
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('FARMACEUTICA'))), 'cod_cbo'] = 2234
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2234'))), 'cod_cbo'] = 2234
#5143 - FAXINEIRO OU 5199 SERVIÇOS GERAIS
#saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('SERVICOSGERAIS'))), 'cod_cbo'] = 5143
#saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('SERVICOGERAL'))), 'cod_cbo'] = 5143
# saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXLIMPEZA'))), 'cod_cbo'] = 5143
# saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDELIMPEZA'))), 'cod_cbo'] = 5143
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.contains('LIMPEZA'))), 'cod_cbo'] = 5143
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.contains('HIGIENIZACAO'))), 'cod_cbo'] = 5143
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.contains('FAXINEIRO'))), 'cod_cbo'] = 5143
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.contains('FAXINEIRA'))), 'cod_cbo'] = 5143
#saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('LAVANDERIA'))), 'cod_cbo'] = 5143
#saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ZELADOR'))), 'cod_cbo'] = 5143
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('MANUTENCAO'))), 'cod_cbo'] = 5143
#saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXGERAL'))), 'cod_cbo'] = 5143
#saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIAROPERACIONAL'))), 'cod_cbo'] = 5143
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARHIGIENE'))), 'cod_cbo'] = 5143
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5143'))), 'cod_cbo'] = 5143
#5151 OU AGENTE DE SAÚDE OU 3522
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ACE'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ACS'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTECOMUNITARIO'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTECOMUNITARIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEDASAUDE'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEDESAUDE'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTESAUDE'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEINDIGENADESAUDE'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTESDEENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEDEENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGDEENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGDENGUE'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTECONTROLEDEENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEDENGUE'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEDECOMBATEAENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEDECOMBATEENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEDECOMBATEASENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEDECOMBATEDEENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTECOMUNITARIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ASSISTENTECOMUNITARIADEENDEMIA'))), 'cod_cbo'] = 5151
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5151'))), 'cod_cbo'] = 5151
#2236 - FISIOTERAPEUTA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('FISIOTERAPEUTA'))), 'cod_cbo'] = 2236
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2236'))), 'cod_cbo'] = 2236
#2232 - DENTISTA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('DENTISTA'))), 'cod_cbo'] = 2232
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('CIRURGIAODENTISTA'))), 'cod_cbo'] = 2232
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('CIRURGIADENTISTA'))), 'cod_cbo'] = 2232
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2232'))), 'cod_cbo'] = 2232
#3224 - AUXILIAR DE ODONTOLOGIA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECEMSAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECDESAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOEMSAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICAEMSAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOODONTO'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICAODONTO'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARODONTO'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIAREMODONTO'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDEODONTO'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDENTISTA'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDEDENTISTA'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARSAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIAREMSAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDESAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXODONTO'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXEMODONTO'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDEODONTO'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXSAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXEMSAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDESAUDEBUCAL'))), 'cod_cbo'] = 3224
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('3224'))), 'cod_cbo'] = 3224
#5134 - COPEIRA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('COPEIRO'))), 'cod_cbo'] = 5134
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('COPEIRA'))), 'cod_cbo'] = 5134
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5134'))), 'cod_cbo'] = 5134
#7823 - MOTORISTA/CONDUTOR
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('CONDUTOR'))), 'cod_cbo'] = 7823
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('MOTORISTA'))), 'cod_cbo'] = 7823
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('7823'))), 'cod_cbo'] = 7823
#5162 - CUIDADOR DE IDOSOS
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('CUIDADOR'))), 'cod_cbo'] = 5162
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('CUIDADORA'))), 'cod_cbo'] = 5162
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5162'))), 'cod_cbo'] = 5162
#4110 - AUXILIAR ADMNISTRATIVO OU 2521?
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ASSADM'))), 'cod_cbo'] = 4110
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ASSISTADM'))), 'cod_cbo'] = 4110
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEADM'))), 'cod_cbo'] = 4110
#saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ADM'))), 'cod_cbo'] = 4110
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXADM'))), 'cod_cbo'] = 4110
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDEESCRITORIO'))), 'cod_cbo'] = 4110
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARADM'))), 'cod_cbo'] = 4110
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARADMINISTRATIVO'))), 'cod_cbo'] = 4110
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ASSISTENTEADM'))), 'cod_cbo'] = 4110
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('4110'))), 'cod_cbo'] = 4110
#2515 - PSICOLOGO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('PSICOLOGA'))), 'cod_cbo'] = 2515
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('PSICOLOGO'))), 'cod_cbo'] = 2515
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('PSICANALISTA'))), 'cod_cbo'] = 2515
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2515'))), 'cod_cbo'] = 2515
#4221 - RECEPCIONISTA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('RECEPCIONISTA'))), 'cod_cbo'] = 4221
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('RECEPCAO'))), 'cod_cbo'] = 4221
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('4221'))), 'cod_cbo'] = 4221
#5152 - AUXILIAR DE LABORATORIO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ASSISTENTEDELABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECEMLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECDELABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOEMLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICAEMLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICADELABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICODELABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICALABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIAREMLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDELABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIAREMLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDELABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXEMLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDELABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXEMLABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDELABORATORIO'))), 'cod_cbo'] = 5152
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5152'))), 'cod_cbo'] = 5152
#2410 - ADVOGADO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ADVOGAD'))), 'cod_cbo'] = 2410
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2410'))), 'cod_cbo'] = 2410
#2237 - NUTIRICIONISTA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('NUTRICIONISTA'))), 'cod_cbo'] = 2237
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2237'))), 'cod_cbo'] = 2237
#2516 - ASSISTENTE SOCIAL
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ASSISTENTESOCIAL'))), 'cod_cbo'] = 2516
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2516'))), 'cod_cbo'] = 2516
#7664 - TECNICO OU AUXILIAR DE RADIOLOGIA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECEMRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECDERADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOEMRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICAEMRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICORADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICARADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIAREMRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDERADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIAREMRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXILIARDERADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXEMRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDERADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXEMRADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDERADIOLOGIA'))), 'cod_cbo'] = 7664
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('7664'))), 'cod_cbo'] = 7664
#2212 - BIOMEDICO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('BIOMEDICO'))), 'cod_cbo'] = 2212
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('BIOMEDICA'))), 'cod_cbo'] = 2212
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2212'))), 'cod_cbo'] = 2212
#4141 - ALMOXARIFE
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ALMOXARIF'))), 'cod_cbo'] = 4141
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('4141'))), 'cod_cbo'] = 4141
#5173 - AGENTE DE SEGURANÇA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AGENTEDESEGURANCA'))), 'cod_cbo'] = 5173
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5173'))), 'cod_cbo'] = 5173
#5211 - ATENDENTE DE FARMACIA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ATENDENTEDEFARMACIA'))), 'cod_cbo'] = 5211
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ATENDENTEFARMACIA'))), 'cod_cbo'] = 5211
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ATENDENTEEMFARMACIA'))), 'cod_cbo'] = 5211
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5211'))), 'cod_cbo'] = 5211
#5132 - COZINHEIRO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('COZINHEIRO'))), 'cod_cbo'] = 5132
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('COZINHEIRA'))), 'cod_cbo'] = 5132
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5132'))), 'cod_cbo'] = 5132
#5135 - AUXILIAR DE COZINHA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXCOZINHA'))), 'cod_cbo'] = 5135
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('AUXDECOZINHA'))), 'cod_cbo'] = 5135
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5135'))), 'cod_cbo'] = 5135
#2233 - VETERINARIO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('VETERINARIO'))), 'cod_cbo'] = 2233
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('VETERINARIA'))), 'cod_cbo'] = 2233
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('MEDICOVETERINARIO'))), 'cod_cbo'] = 2233
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('MEDICAVETERINARIA'))), 'cod_cbo'] = 2233
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2233'))), 'cod_cbo'] = 2233
#2238 _ FONOAUDIOLOGIA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('FONOAUDIOLOGA'))), 'cod_cbo'] = 2238
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('FONOAUDIOLOGO'))), 'cod_cbo'] = 2238
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2238'))), 'cod_cbo'] = 2238
#3516 - SEGURANCA DO TRABALHO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECEMSEGURANCADOTRABALHO'))), 'cod_cbo'] = 3516
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECDESEGURANCADOTRABALHO'))), 'cod_cbo'] = 3516
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICOEMSEGURANCADOTRABALHO'))), 'cod_cbo'] = 3516
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICAEMSEGURANCADOTRABALHO'))), 'cod_cbo'] = 3516
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICODESEGURANCADOTRABALHO'))), 'cod_cbo'] = 3516
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('TECNICADESEGURANCADOTRABALHO'))), 'cod_cbo'] = 3516
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('3516'))), 'cod_cbo'] = 3516
#2132 _ QUIMICO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('QUIMICA'))), 'cod_cbo'] = 2132
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('QUIMICO'))), 'cod_cbo'] = 2132
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2132'))), 'cod_cbo'] = 2132
#2241 _ EDUCADOR FISICO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('EDUCADORFISICO'))), 'cod_cbo'] = 2241
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('PERSONALTRAINER'))), 'cod_cbo'] = 2241
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2241'))), 'cod_cbo'] = 2241
#5171 _ SOCORRISTA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('SOCORRISTA'))), 'cod_cbo'] = 5171
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5171'))), 'cod_cbo'] = 5171
#2131 - FISICO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('FISICO'))), 'cod_cbo'] = 2131
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2131'))), 'cod_cbo'] = 2131
#3221 - ESTETICISTA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ESTETICISTA'))), 'cod_cbo'] = 3221
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('3221'))), 'cod_cbo'] = 3221
#2521 - ADMINISTRADOR
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ADMINISTRADOR'))), 'cod_cbo'] = 2521
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2521'))), 'cod_cbo'] = 2521
#1421 - GERENTE
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('GERENTE'))), 'cod_cbo'] = 1421
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('1421'))), 'cod_cbo'] = 1421
#1231 - DIRETOR
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('DIRETOR'))), 'cod_cbo'] = 1231
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('1231'))), 'cod_cbo'] = 1231
#5141 - ZELADOR
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('ZELADOR'))), 'cod_cbo'] = 5141
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5141'))), 'cod_cbo'] = 5141
#2132 - QUIMICO
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('QUIMICA'))), 'cod_cbo'] = 2132
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('QUIMICO'))), 'cod_cbo'] = 2132
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('2132'))), 'cod_cbo'] = 2132
#5174 - VIGIA
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('VIGIA'))), 'cod_cbo'] = 5174
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('VIGILANTE'))), 'cod_cbo'] = 5174
saude.loc[((saude['cbo_descricao'] == 'Não informado') & (saude['hash_descricao_ocupacao'].str.startswith('5174'))), 'cod_cbo'] = 5174

In [ ]:
tabela_not = saude.groupby('cbo_descricao')[['paciente']].count().rename(columns={'paciente':'Notificados'}).sort_values(by=['Notificados'], ascending = False).reset_index()
tabela_conf = saude.loc[saude['classificacao_final']==2].groupby('cbo_descricao')[['paciente']].count().rename(columns={'paciente':'Confirmados'}).sort_values(by=['Confirmados'], ascending = False).reset_index()
tabela_rec = saude.loc[(saude['evolucao']==1) & (saude['classificacao_final']==2)].groupby('cbo_descricao')[['paciente']].count().rename(columns={'paciente':'Recuperados'}).sort_values(by=['Recuperados'], ascending = False).reset_index()
tabela_obt = saude.loc[(saude['evolucao']==2) & (saude['classificacao_final']==2)].groupby('cbo_descricao')[['paciente']].count().rename(columns={'paciente':'Óbitos'}).sort_values(by=['Óbitos'], ascending = False).reset_index()

In [ ]:
tabela = pd.merge(left=tabela_conf, right=tabela_rec, on='cbo_descricao',how='left')
tabela = pd.merge(left=tabela, right=tabela_obt, on='cbo_descricao',how='left')
tabela = pd.merge(left=tabela, right=tabela_not, on='cbo_descricao',how='left')
tabela = tabela.sort_values(by=['Confirmados'],  ascending = False)
tabela = tabela.rename(columns={'cbo_descricao':'Ocupação'})

In [ ]:
tabela.to_excel(join(default_output, 'semanais', 'TABELA_trabalhadores_saude_novo.xlsx'), index= False)

In [ ]:
del saude['cbo_descricao']
saude['cod_cbo'] = saude['cod_cbo'].astype('int64')
saude = saude.merge(how = 'left', on = 'cod_cbo', right = cbo)
saude.loc[(saude['cbo_descricao'] == 'Não informado') & (saude['cod_cbo'] == 0) & (saude['hash_descricao_ocupacao'].isnull())].sort_values(['cod_cbo', 'hash_descricao_ocupacao'])[['id', 'data_notificacao', 'ocupacao_descricao', 'cod_cbo', 'cbo_descricao']].rename(columns={'id': 'ID_NOTIFICA-COVID19', 'data_notificacao':'DATA_NOTIFICACAO', 'ocupacao_descricao':'DESCRICAO_DA_OCUPACAO', 'cod_cbo':'CODIGO_CBO_SUGERIDO', 'cbo_descricao':'DESCRICAO_CODIGO_CBO'}).to_excel(join(default_output, 'semanais', 'EXPORTACAO_descricao_vazia.xlsx'), index= False)
saude.loc[(saude['cbo_descricao'] == 'Não informado') & (saude['cod_cbo'] == 0) & (saude['hash_descricao_ocupacao'].isnull())].shape[0]

In [ ]:
saude.loc[(saude['cbo_descricao'] == 'Não informado') & (saude['cod_cbo'] != 0) & (~saude['hash_descricao_ocupacao'].isnull())].sort_values(['cod_cbo', 'hash_descricao_ocupacao'])[['id', 'data_notificacao', 'ocupacao_descricao', 'cod_cbo', 'cbo_descricao']].rename(columns={'id': 'ID_NOTIFICA-COVID19', 'data_notificacao':'DATA_NOTIFICACAO', 'ocupacao_descricao':'DESCRICAO_DA_OCUPACAO', 'cod_cbo':'CODIGO_CBO_SUGERIDO', 'cbo_descricao':'DESCRICAO_CODIGO_CBO'}).to_excel(join(default_output, 'semanais', 'EXPORTACAO_verificar_cbo_qualificados.xlsx'), index= False)
saude.loc[(saude['cbo_descricao'] == 'Não informado') & (saude['cod_cbo'] != 0) & (~saude['hash_descricao_ocupacao'].isnull())].shape[0]

In [ ]:
saude.loc[(saude['cbo_descricao'] == 'Não informado') & (saude['cod_cbo'] == 0) & (~saude['hash_descricao_ocupacao'].isnull())].sort_values(['cod_cbo', 'hash_descricao_ocupacao'])[['id', 'data_notificacao', 'ocupacao_descricao', 'cod_cbo', 'cbo_descricao']].rename(columns={'id': 'ID_NOTIFICA-COVID19', 'data_notificacao':'DATA_NOTIFICACAO', 'ocupacao_descricao':'DESCRICAO_DA_OCUPACAO', 'cod_cbo':'CODIGO_CBO_SUGERIDO', 'cbo_descricao':'DESCRICAO_CODIGO_CBO'}).to_excel(join(default_output, 'semanais', 'EXPORTACAO_verificar_cbo_para_qualificar.xlsx'), index= False)
saude.loc[(saude['cbo_descricao'] == 'Não informado') & (saude['cod_cbo'] == 0) & (~saude['hash_descricao_ocupacao'].isnull())].shape[0]

In [ ]:
tabela